In [1]:
from urllib.request import urlopen
from urllib.parse import quote_plus as qp #문자 아스키코드로 변환
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import os

In [2]:
#인터넷 창 띄우기(chrome driver에 각자 개별 로그인)
driver = webdriver.Chrome()
url = 'https://www.instagram.com/'
driver.get(url)

In [3]:
#검색어 태그로 인터넷 창 이동
search = '헤걀라드' #검색어
baseurl = 'https://www.instagram.com/explore/tags/'
url = baseurl + qp(search)
driver.get(url)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME,'v1Nh3.kIKUG._bz0w')))
imgurl=[] #이미지 url 저장 list

In [4]:
scroll_num = 100 #스크롤 횟수
scroll_pause_time=1.5 #스크롤 내릴때 이미지 로딩되는 시간 기다림
upscroll=-100
#스크롤 하기
#스크롤은 기다리기 귀찮으면 직접 한 후에 아래 코드 실행해도 됨
#인터넷 환경에 따라 이미지 로딩이 느릴때, 스크롤이 적게 실행될 수 있음
breaker = [] # 업데이트 5회이상 안될시, break


for i in range(scroll_num+1):

    #파싱
    html = driver.page_source
    soup = BeautifulSoup(html)
    insta = soup.select('.v1Nh3.kIKUG._bz0w')
    for pic in insta:
        imgurl.append(pic.select_one('.KL4Bh').img['src']) #이미지 주소 가져오기
        imgurl = (list(set(imgurl))) #이미지주소 중복 제거
    
    print('스크롤 {}번 실행, {}개 그림을 가져올 수 있습니다.'.format(i, len(list(set(imgurl)))))
    breaker.append(len(list(set(imgurl))))
    time.sleep(scroll_pause_time)
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script('scrollBy(0,{})'.format(upscroll))
    
    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script('scrollBy(0,{})'.format(upscroll))
        if new_height == last_height:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)
            new_height = driver.execute_script("return document.body.scrollHeight")
            driver.execute_script('scrollBy(0,{})'.format(upscroll))
        else:
            last_height = new_height
            continue
    if len(breaker)>5 and breaker[-5]==breaker[-1]: 
        print('더이상 update가 없습니다.')
        break

스크롤 0번 실행, 33개 그림을 가져올 수 있습니다.
스크롤 1번 실행, 45개 그림을 가져올 수 있습니다.
스크롤 2번 실행, 57개 그림을 가져올 수 있습니다.
스크롤 3번 실행, 69개 그림을 가져올 수 있습니다.
스크롤 4번 실행, 72개 그림을 가져올 수 있습니다.
스크롤 5번 실행, 72개 그림을 가져올 수 있습니다.
스크롤 6번 실행, 72개 그림을 가져올 수 있습니다.


WebDriverException: Message: chrome not reachable
  (Session info: chrome=101.0.4951.67)
Stacktrace:
Backtrace:
	Ordinal0 [0x00D2B8F3+2406643]
	Ordinal0 [0x00CBAF31+1945393]
	Ordinal0 [0x00BAC610+837136]
	Ordinal0 [0x00BA0442+787522]
	Ordinal0 [0x00BA0C78+789624]
	Ordinal0 [0x00BA24B2+795826]
	Ordinal0 [0x00B9BF09+769801]
	Ordinal0 [0x00BADAC0+842432]
	Ordinal0 [0x00C03E62+1195618]
	Ordinal0 [0x00BF4096+1130646]
	Ordinal0 [0x00BCE636+976438]
	Ordinal0 [0x00BCF546+980294]
	GetHandleVerifier [0x00F99612+2498066]
	GetHandleVerifier [0x00F8C920+2445600]
	GetHandleVerifier [0x00DC4F2A+579370]
	GetHandleVerifier [0x00DC3D36+574774]
	Ordinal0 [0x00CC1C0B+1973259]
	Ordinal0 [0x00CC6688+1992328]
	Ordinal0 [0x00CC6775+1992565]
	Ordinal0 [0x00CCF8D1+2029777]
	BaseThreadInitThunk [0x75C4FA29+25]
	RtlGetAppContainerNamedObjectPath [0x76FA7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x76FA7A4E+238]


In [125]:
if search not in os.listdir(): os.mkdir(search)  #폴더 생성
for i, pic in enumerate(imgurl):
    with urlopen(pic) as f: #날짜 시간 포함하여 이미지 저장
        with open('./'+search+'/'+search+'_'+'{0:05d}'.format(i+1)+'_'+time.strftime('%y%m%d-%H%M')+'.jpg','wb') as h:
            img = f.read()
            h.write(img)
    print('이미지 저장 중 : {}/{}'.format(i+1,len(imgurl)), end='\r', flush=True)